In [337]:
import numpy as np
import matplotlib.pyplot as plt

In [338]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=5000)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
5000
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [339]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
            WIN[inp][new_index] += 1
        else:
            new_index = max(IdxO, key=IdxO.get)

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX


max_index_L1 = 32
max_history_L1 = 7

DSHistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

for h_idx in range(max_history_L1):
    for wk, wv in DSHistoryW[h_idx].items():
        print(wk, wv)


c0 {'h': {0: 1}}
h0 {'a': {0: 2}, 'o': {4: 3}, 'i': {18: 2, 30: 1}, 'e': {12: 4}, 't': {1: 2}, ' ': {7: 3, 3: 1}}
a0 {'p': {0: 2}, 'i': {2: 5, 7: 1}, 'y': {2: 1}, 'm': {20: 1}, 'k': {7: 1}, 'v': {8: 1}, 'w': {28: 1}, 'n': {4: 1}, 'l': {14: 1}, ' ': {2: 1}, 'd': {31: 2}}
p0 {'t': {0: 1}, 'e': {6: 1}, 'p': {10: 1}}
t0 {'e': {0: 1}, 'o': {30: 1}, 'h': {5: 3}, 'i': {0: 1}}
e0 {'r': {0: 4}, 'l': {6: 2}, 's': {29: 1}, ' ': {25: 4}, 't': {1: 1}, 'd': {1: 1}, 'n': {26: 1}}
r0 {' ': {0: 4, 17: 2}, 'y': {10: 1}, 'e': {24: 3}}
 0 {'i': {0: 3}, 'b': {7: 1}, 'h': {20: 1, 1: 1}, 'a': {22: 1}, 's': {31: 1}, 'w': {17: 1}}
i0 {'n': {0: 2}, 'd': {19: 2}, 'c': {10: 1}, ' ': {1: 3}, 't': {22: 3, 27: 1}, 'k': {9: 2}, 'm': {29: 1}, 's': {13: 1, 26: 2}, 'f': {1: 1}, 'g': {0: 1}}
n0 {' ': {1: 4, 17: 2}, 'y': {5: 1}, 'o': {5: 1}, 'd': {3: 3}, 'n': {6: 1}, 'e': {12: 1}, 'k': {13: 1}}
 1 {'w': {0: 2}, 'c': {5: 5}, 'f': {7: 4, 3: 1}, 'h': {2: 7}, 'a': {23: 2, 0: 1, 4: 1}, 's': {9: 2, 3: 2}, 'n': {20: 2}, 'b': {18

In [340]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    AL = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        max_arr = []
        for v in HO.values():
            col_max = max(v, key=v.get)
            max_arr.append(col_max)
        
        max_arr = np.array(max_arr)
        # tf = np.mean(np.abs(max_arr - max_arr[np.newaxis].T))

        tf = length
        # tf = len(HO)

        TF.append(tf)

        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}

            if val[max_idx] > max_val:
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=}', max_symbol_index, max_val)

        # anomaly
        if max_symbol == next_inp:
            AL.append(0)
        else:
            ni_v = HO.get(next_inp, 0)
            if ni_v == 0:
                AL.append(100)
            else:
                max_ni = max(ni_v, key=ni_v.get)
                AL.append(abs(ni_v[max_ni] - max_val))

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    TF.append(0)
    AL.append(0)
    return err, TF, AL

err_L1, TF_L1, AL_L1 = predict(input_str, DSHistoryW, max_index=max_index_L1, max_history=max_history_L1)

print(err_L1, err_L1 / (len(input_str) / 100))

max_symbol='h' next_inp='h' 0 3
max_symbol=' ' next_inp='a' 7 6
max_symbol='p' next_inp='p' 0 5
max_symbol='d' next_inp='t' 24 7
max_symbol='e' next_inp='e' 0 8
max_symbol='r' next_inp='r' 0 9
max_symbol=' ' next_inp=' ' 0 14
max_symbol='i' next_inp='i' 0 15
max_symbol='n' next_inp='n' 0 10
max_symbol=' ' next_inp=' ' 1 28
max_symbol='w' next_inp='w' 0 12
max_symbol='h' next_inp='h' 1 23
max_symbol='e' next_inp='i' 14 17
max_symbol=' ' next_inp='c' 19 16
max_symbol='h' next_inp='h' 2 8
max_symbol=' ' next_inp=' ' 2 17
max_symbol=' ' next_inp='w' 16 14
max_symbol='e' next_inp='e' 1 12
max_symbol=' ' next_inp=' ' 3 14
max_symbol='a' next_inp='a' 1 9
max_symbol='r' next_inp='r' 1 9
max_symbol='e' next_inp='e' 2 8
max_symbol=' ' next_inp=' ' 4 8
max_symbol='i' next_inp='i' 2 8
max_symbol='n' next_inp='n' 1 12
max_symbol=' ' next_inp='t' 5 21
max_symbol='h' next_inp='r' 15 14
max_symbol='e' next_inp='o' 11 11
max_symbol=' ' next_inp='d' 6 19
max_symbol=' ' next_inp='u' 16 20
max_symbol='n' 

In [341]:
def get_word_stats(input_str: str | list, TF: list, threshold=0.0):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}
    WordIndex: dict[str, int] = {}
    next_wrd_idx = 0

    NextLayerOut = []
    DownstreamIndex = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 and inp_idx >= len(input_str) - 1:
            continue

        # if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
        # and TF[inp_idx] > threshold:
        if TF[inp_idx] > prev_tf and TF[inp_idx] > TF[inp_idx + 1]:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            if not WordIndex.get(word):
                WordIndex.setdefault(word, next_wrd_idx)
                next_wrd_idx += 1

            wrd_idx = WordIndex.get(word)
            # NextLayerOut.append(str(wrd_idx))
            NextLayerOut.append(word)

            DownstreamIndex.setdefault(inp_idx, 1)

            buffer = [inp]
            prev_tf = TF[inp_idx]

            continue

        TF[inp_idx] = 0

        buffer.append(inp)
        prev_tf = TF[inp_idx]

    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    NextLayerOut.append(str(next_wrd_idx))
    return WordStats, WordIndex, NextLayerOut, DownstreamIndex


TF_L1 = np.array(TF_L1, dtype='float64')
AL_L1 = np.array(AL_L1, dtype='float64')
TF_L1 /= np.max(TF_L1)
AL_L1 /= np.max(AL_L1)
TF_L1 += AL_L1

TF_max_coef = 0.4

# TF_L1[np.where(TF_L1 < np.mean(TF_L1))] = 0
# TF_1 = TF_L1 - np.pad(TF_L1, (1, 0))[:-1]
# TF_2 = TF_L1 - np.pad(TF_L1, (0, 1))[1:]
# TF_L1 = TF_1 + TF_2

# TF_L1[np.where(TF_L1 < 0)] = 0
# TF_L1[np.where(TF_L1 < TF_max_coef)] = 0
TF_L1 = TF_L1.tolist()

WordStats_L1, WordIndex_L1, NextLayerOut_L1, DownstreamIndex_L1 = get_word_stats(input_str=input_str,
                                                                                 TF=TF_L1, 
                                                                                 threshold=0.0, )
print(len(WordStats_L1))
mean_WS = np.mean(np.array(list(WordStats_L1.values())))
print(f'{mean_WS=}')
WS_len = 0
for key, val in WordStats_L1.items():
    # if len(key.strip()) > 1 and val >= mean_WS:
    if val >= mean_WS:
        WS_len += 1
    print(key, val)
print(f'{WS_len=}')

print(f'{NextLayerOut_L1=}')
print(f'{len(NextLayerOut_L1)=}')

print(f'{WordIndex_L1=}')

print(f'{DownstreamIndex_L1=}')


753
mean_WS=2.349269588313413
 t 35
 a 27
 h 27
 b 27
ng 22
 w 20
ou 20
 y 18
 p 17
nd 17
 s 16
he 16
on 15
er 15
 f 14
e  13
es 13
 he 13
 l 13
in 12
 th 12
 c 12
 i 11
hat 11
the 10
is 10
ow 10
to 10
 n 10
 g 10
 the 10
as 9
im 9
sa 9
id 9
 to 8
 d 8
en 8
ed 8
 o 8
ni 7
bu 7
hi 7
 he  7
 to  7
al 7
it 7
ad 7
be 6
ut 6
at 6
ed  6
ooh 6
 robin 6
se 6
 so 6
 m 6
or 6
 be 5
om 5
et 5
ll 5
ea 5
ir 5
st 5
an 5
't 5
s  5
 po 5
d  5
of 5
ry 5
hat  5
re 5
cl 5
ty 5
lo 5
 wh 4
w  4
 bu 4
pher 4
oth 4
 i  4
 as 4
all 4
ie  4
th 4
oh 4
 do 4
he  4
et  4
nd  4
 of  4
out 4
elf 4
ee 4
t  4
en  4
ing 4
tr 4
of  4
zz 4
and 4
g  4
ey 4
 it 4
his 4
er  4
if 4
ra 4
ere  3
sto 3
as  3
ly 3
ay  3
irs 3
y  3
ly  3
 co 3
 for 3
nk 3
 his 3
ght 3
aid 3
hr 3
winn 3
h  3
 it  3
oi 3
ik 3
so 3
me 3
 fr 3
 li 3
ld 3
ai 3
 of 3
ime 3
 ve 3
 lo 3
ti 3
iv 3
un 3
 wa 3
aid  3
 sa 3
n  3
zi 3
uz 3
mak 3
 re 3
ho 3
 and 3
imb 3
nt 3
 a b 3
do 3
fu 3
ug 3
at  3
we 3
el 3
on  3
her 3
r  2
are 2
ced 2
ooh  2
ome 2
ori 2

In [342]:
def downstream_train(input: list, next_layer_out: list, downstream_index: dict[int, int]):
    DSStats = {}

    ds_idx = 1
    ds_inp = next_layer_out[0]

    for inp_idx, inp in enumerate(input[:-1]):
        has_ds_input = downstream_index.get(inp_idx)
        if has_ds_input:
            ds_inp = next_layer_out[ds_idx]
            ds_idx += 1

        ds_key = ds_inp + ':' + inp
        DSStats.setdefault(ds_key, 0)
        DSStats[ds_key] += 1

    return DSStats


# DSStats = downstream_train(input=input_str,
#                            next_layer_out=NextLayerOut_L1,
#                            downstream_index=DownstreamIndex_L1,)

# for wk, wv in DSStats.items():
#     print(wk, wv)

In [343]:
def downstream_predict_train(input_str: str,
                             upstream_inp: dict[int, list],
                             ds_input: list,
                             max_history=4,
                             max_index=4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    ds_idx = 1
    ds_inp = ds_input[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
            WIN[inp][new_index] += 1
        else:
            new_index = max(IdxO, key=IdxO.get)

        # ds input
        has_ds_input = upstream_inp.get(inp_idx - 1)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = ds_inp + ':' + prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX


HistoryW_L1_ds, HistoryHW_L1_ds, IDX_L1_ds = downstream_predict_train(input_str=input_str,
                                                                      upstream_inp=DownstreamIndex_L1,
                                                                      ds_input=NextLayerOut_L1,
                                                                      max_history=max_history_L1,
                                                                      max_index=max_index_L1,)

for h_idx in range(max_history_L1):
    for wk, wv in HistoryW_L1_ds[h_idx].items():
        print(wk, wv)


ccha:c0 {'h': {0: 1}}
ccha:h0 {'a': {0: 1}}
ccha:a0 {'p': {0: 1}}
pte:p0 {'t': {0: 1}}
pte:t0 {'e': {0: 1}}
pte:e0 {'r': {0: 1}}
r :r0 {' ': {0: 1}}
r : 0 {'i': {0: 1}}
in:i0 {'n': {0: 1}}
in:n0 {' ': {1: 1}}
 wh: 1 {'w': {0: 1}}
 wh:w0 {'h': {1: 1}}
 wh:h1 {'i': {1: 1}}
ich:i1 {'c': {1: 1}}
ich:c1 {'h': {2: 1}}
ich:h2 {' ': {2: 1}}
 w: 2 {'w': {1: 1}}
 w:w1 {'e': {1: 1}}
e :e1 {' ': {3: 1}}
e : 3 {'a': {1: 1}}
are:a1 {'r': {1: 1}}
are:r1 {'e': {2: 1}}
are:e2 {' ': {4: 1}}
 i: 4 {'i': {2: 1}}
 i:i2 {'n': {1: 1}}
ntro:n1 {'t': {1: 1}}
ntro:t1 {'r': {2: 1}}
ntro:r2 {'o': {0: 1}}
ntro:o0 {'d': {0: 1}}
du:d0 {'u': {0: 1}}
du:u0 {'c': {2: 1}}
ced:c2 {'e': {3: 2}}
ced:e3 {'d': {1: 2}}
ced:d1 {' ': {5: 2}}
 to: 5 {'t': {2: 2}}
 to:t2 {'o': {1: 1, 5: 1}}
 to:o1 {' ': {6: 1}}
 win: 6 {'w': {2: 1}}
 win:w2 {'i': {3: 1}}
 win:i3 {'n': {2: 1}}
 win:n2 {'n': {3: 1}}
ni:n3 {'i': {4: 1}}
ni:i4 {'e': {4: 1}}
e :e4 {' ': {7: 2}}
e : 7 {'t': {3: 1}, 'c': {16: 1}}
the:t3 {'h': {3: 2}}
the:h3 {'e': {5: 2}

In [344]:
def downstream_predict(input_str: str,
                       DSHistoryW: list[dict[str, dict[str, dict[int, int]]]],
                       upstream_inp: dict[int, list],
                       ds_input: list,
                       max_history=4,
                       max_index=4,):
    err = 0
    TF = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    ds_idx = 1
    ds_inp = ds_input[0]

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        # ds input
        has_ds_input = upstream_inp.get(input_idx)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = DSHistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = ds_inp + ':' + inp + str(indices[0])
            # print(f'{key_I=}')
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = ds_inp + ':' + inp + str(index)
                # print(f'{key_I=}')

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)

            # print(f'{O=}')
            # print()

            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})

                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        # max symbol
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}
            max_idx_val = val[max_idx]

            # ds predict
            # ds_key = ds_inp + ':' + key
            # ds_stats = DSStats.get(ds_key)
            # if ds_stats:
            #     max_idx_val += ds_stats

            if max_idx_val > max_val: # and ds_stats:
                max_val = max_idx_val
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=} {ds_inp=} {HO=}',
              max_symbol_index, max_val)

        # print(f'{O.get(max_symbol).keys()=}')
        new_indices = [max_symbol_index]

        if O.get(max_symbol):
            new_indices = list(O.get(max_symbol).keys())

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)

            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]

        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF


err_L1_ds, TF_L1_ds = downstream_predict(input_str=input_str,
                                         DSHistoryW=HistoryW_L1_ds,
                                         upstream_inp=DownstreamIndex_L1,
                                         ds_input=NextLayerOut_L1,
                                         max_history=max_history_L1,
                                         max_index=max_index_L1, )

print(f'{err_L1=}', err_L1 / (len(input_str) / 100))
print(f'{err_L1_ds=}', err_L1_ds / (len(input_str) / 100))


max_symbol='h' next_inp='h' ds_inp='ccha' HO={'h': {0: 1}} 0 1
max_symbol='a' next_inp='a' ds_inp='ccha' HO={'a': {0: 2}} 0 2
max_symbol='p' next_inp='p' ds_inp='ccha' HO={'p': {0: 3}} 0 3
max_symbol='t' next_inp='t' ds_inp='pte' HO={'t': {0: 4}} 0 4
max_symbol='e' next_inp='e' ds_inp='pte' HO={'e': {0: 5}} 0 5
max_symbol='r' next_inp='r' ds_inp='pte' HO={'r': {0: 6}} 0 6
max_symbol=' ' next_inp=' ' ds_inp='r ' HO={' ': {0: 7}} 0 7
max_symbol='i' next_inp='i' ds_inp='r ' HO={'i': {0: 7}} 0 7
max_symbol='n' next_inp='n' ds_inp='in' HO={'n': {0: 7}} 0 7
max_symbol=' ' next_inp=' ' ds_inp='in' HO={' ': {1: 7}} 1 7
max_symbol='w' next_inp='w' ds_inp=' wh' HO={'w': {0: 7}} 0 7
max_symbol='h' next_inp='h' ds_inp=' wh' HO={'h': {1: 7}} 1 7
max_symbol='i' next_inp='i' ds_inp=' wh' HO={'i': {1: 7}} 1 7
max_symbol='c' next_inp='c' ds_inp='ich' HO={'c': {1: 7}} 1 7
max_symbol='h' next_inp='h' ds_inp='ich' HO={'h': {2: 7}} 2 7
max_symbol=' ' next_inp=' ' ds_inp='ich' HO={' ': {2: 7}} 2 7
max_symbo